In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-01-01'
final_date = '2022-01-31'
query_error_year = 2022
query_error_month = 1

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\01. ENERO\DATA\ERROR ENERO 2022.csv',dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 10)
#                 & (query_error['year'] == 2021)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 10 )
#             & (query_error['year'] == 2021), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15 22:01:46,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15 22:02:08,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01 19:53:09,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12 17:07:39,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12 17:07:25,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(67558, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
57622,57622,b42fc20a-c041-4dd3-a740-3c78c85c31a5,Carniceria Local 21,RQL2J4GGS,2021-12-31,"1,348.25",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:10,1,2022,NEW,ERROR_USUAL,b42fc20a-c041-4dd3-a740-3c78c85c31a5,2022-01-01,1,Actual
57506,57506,12a5d534-e735-46d0-88f7-86430a745e39,Umedica,9ZSJ5KHXS,2021-12-31,143.74,ERROR,DEV.SPEICUENTA CANCELADA,2022-01-01 19:09:54,1,2022,NEW,ERROR_USUAL,12a5d534-e735-46d0-88f7-86430a745e39,2022-01-01,1,Actual
57507,57507,3cf78e3d-91b1-42b7-8be5-759d04f6fcba,Tienda RD,QBDJXR8NC,2021-12-31,80.90,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:14,1,2022,NEW,ERROR_USUAL,3cf78e3d-91b1-42b7-8be5-759d04f6fcba,2022-01-01,1,Actual
57508,57508,4dd0d5fc-7d15-4777-96e4-2dec974e7eee,Gelatto,YRSMQ5PBL,2021-12-31,"1,026.27",ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-01-01 19:09:55,1,2022,NEW,ERROR_USUAL,4dd0d5fc-7d15-4777-96e4-2dec974e7eee,2022-01-01,1,Actual
57510,57510,c56fedc0-285e-4cdb-83db-79aa3f683507,Frutas,XXYVQSMWX,2021-12-31,"1,290.73",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:05,1,2022,NEW,ERROR_USUAL,c56fedc0-285e-4cdb-83db-79aa3f683507,2022-01-01,1,Actual


In [9]:
query_error_current_month.shape

(10470, 17)

In [10]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [11]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [12]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [13]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
16072,16072,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,XB4QJ9WRZ,2021-02-21,178.23,ERROR,Cuenta cancelada,2021-12-01 15:38:03,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN
16133,16133,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,HDBBSP5J7,2021-02-22,111.15,ERROR,Cuenta cancelada,2021-12-01 15:38:09,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN
16807,16807,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,VFGMG6F67,2021-02-27,354.55,ERROR,Cuenta cancelada,2021-12-01 15:37:54,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN
19196,19196,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,57FSM6FCT,2021-04-30,618.07,ERROR,Cuenta cancelada,2021-12-01 15:37:54,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN


In [14]:
query_new_last_month.shape

(4, 17)

# INFORMACIÓN QUERIES - PAGOS

In [15]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [16]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [17]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [18]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [19]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2022\01. JANUARY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\My Drive\Settlement\FAST PAYMENTS\2022\01. ENERO\QUERIES\*.csv', recursive=False)

In [20]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

In [21]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [22]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\12. DECEMBER\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\My Drive\Settlement\FAST PAYMENTS\2021\12. DICIEMBRE\QUERIES\*.csv', recursive=False)

In [23]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [24]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

## ALL FILES

In [25]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,swap,file_name,date_query,tipo_dispersion
35207,748c78b8-cadc-483e-ae50-60715de44756,Wacho’s Texcoco,rafael_wachos@hotmail.com,2223TPR8N,2021-12-01,"1,097.19","BBVA Bancomer, S.A.",2021-12-01 08:15:57,Jocelyn Martinez Hernandez,'012180015634257038,2021-12-01 08:09:10,"Jocelyn Martinez Hernandez ,,1097.19,012180015...",SWAP 2021-12-01.csv,2021-12-01,TRAD
43773,9150491a-374e-472c-8d33-e94c9f255539,La Batucada,labatucadatoluca@gmail.com,222FZK6KC,2021-12-01,10.00,"Banco Nacional de México, S.A.",2021-12-01 08:15:57,Socorro Elizabeth Chavez Silva,'002420701716193845,2021-12-01 08:10:42,"Socorro Elizabeth Chavez Silva,,10.00,00242070...",SWAP 2021-12-01.csv,2021-12-01,TRAD
30277,6468a3fc-ce8a-45af-9003-bf154ae7a855,Michelle bags,ib_ago@hotmail.com,22365XV7N,2021-12-01,"2,219.28","BBVA Bancomer, S.A.",2021-12-01 08:15:57,Juan Carlos Arguelles Ochoa,'012320001871309102,2021-12-01 08:08:30,"Juan Carlos Arguelles Ochoa,,2219.28,012320001...",SWAP 2021-12-01.csv,2021-12-01,TRAD
26086,564d2389-419a-45ec-a278-c08573ddc6de,Robe Metamorfo,robemetamorfo83@gmail.com,2238N48VN,2021-12-01,"5,749.44","BBVA Bancomer, S.A.",2021-12-01 08:17:00,Roberto Carlos Gómez Ramírez,'012700028365975726,2021-12-01 08:07:54,"Roberto Carlos Gómez Ramírez,,5749.44,01270002...",SWAP 2021-12-01.csv,2021-12-01,TRAD
34224,71681547-a00d-4b43-b366-f59ab2f7578d,Tqs,chemaesp@hotmail.com,223CB4S2K,2021-12-01,266.49,"BBVA Bancomer, S.A.",2021-12-01 08:16:28,Jose maria eparza,'012180015107748443,2021-12-01 08:09:02,"Jose maria eparza,,266.49,012180015107748443,C...",SWAP 2021-12-01.csv,2021-12-01,TRAD


In [26]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [27]:
allfiles.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,swap,file_name,date_query,tipo_dispersion,tipo_pago
35207,748c78b8-cadc-483e-ae50-60715de44756,Wacho’s Texcoco,rafael_wachos@hotmail.com,2223TPR8N,2021-12-01,"1,097.19","BBVA Bancomer, S.A.",2021-12-01 08:15:57,Jocelyn Martinez Hernandez,'012180015634257038,2021-12-01 08:09:10,"Jocelyn Martinez Hernandez ,,1097.19,012180015...",SWAP 2021-12-01.csv,2021-12-01,TRAD,Pago del día
43773,9150491a-374e-472c-8d33-e94c9f255539,La Batucada,labatucadatoluca@gmail.com,222FZK6KC,2021-12-01,10.00,"Banco Nacional de México, S.A.",2021-12-01 08:15:57,Socorro Elizabeth Chavez Silva,'002420701716193845,2021-12-01 08:10:42,"Socorro Elizabeth Chavez Silva,,10.00,00242070...",SWAP 2021-12-01.csv,2021-12-01,TRAD,Pago del día
30277,6468a3fc-ce8a-45af-9003-bf154ae7a855,Michelle bags,ib_ago@hotmail.com,22365XV7N,2021-12-01,"2,219.28","BBVA Bancomer, S.A.",2021-12-01 08:15:57,Juan Carlos Arguelles Ochoa,'012320001871309102,2021-12-01 08:08:30,"Juan Carlos Arguelles Ochoa,,2219.28,012320001...",SWAP 2021-12-01.csv,2021-12-01,TRAD,Pago del día
26086,564d2389-419a-45ec-a278-c08573ddc6de,Robe Metamorfo,robemetamorfo83@gmail.com,2238N48VN,2021-12-01,"5,749.44","BBVA Bancomer, S.A.",2021-12-01 08:17:00,Roberto Carlos Gómez Ramírez,'012700028365975726,2021-12-01 08:07:54,"Roberto Carlos Gómez Ramírez,,5749.44,01270002...",SWAP 2021-12-01.csv,2021-12-01,TRAD,Pago del día
34224,71681547-a00d-4b43-b366-f59ab2f7578d,Tqs,chemaesp@hotmail.com,223CB4S2K,2021-12-01,266.49,"BBVA Bancomer, S.A.",2021-12-01 08:16:28,Jose maria eparza,'012180015107748443,2021-12-01 08:09:02,"Jose maria eparza,,266.49,012180015107748443,C...",SWAP 2021-12-01.csv,2021-12-01,TRAD,Pago del día


In [28]:
allfiles.shape

(4838507, 16)

# CRUCE 1 - MES ACTUAL

In [29]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,748c78b8-cadc-483e-ae50-60715de44756,Wacho’s Texcoco,rafael_wachos@hotmail.com,2223TPR8N,2021-12-01,"1,097.19","BBVA Bancomer, S.A.",2021-12-01 08:15:57,Jocelyn Martinez Hernandez,'012180015634257038,...,TRAD,Pago del día,nan,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,9150491a-374e-472c-8d33-e94c9f255539,La Batucada,labatucadatoluca@gmail.com,222FZK6KC,2021-12-01,10.00,"Banco Nacional de México, S.A.",2021-12-01 08:15:57,Socorro Elizabeth Chavez Silva,'002420701716193845,...,TRAD,Pago del día,nan,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,6468a3fc-ce8a-45af-9003-bf154ae7a855,Michelle bags,ib_ago@hotmail.com,22365XV7N,2021-12-01,"2,219.28","BBVA Bancomer, S.A.",2021-12-01 08:15:57,Juan Carlos Arguelles Ochoa,'012320001871309102,...,TRAD,Pago del día,nan,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,564d2389-419a-45ec-a278-c08573ddc6de,Robe Metamorfo,robemetamorfo83@gmail.com,2238N48VN,2021-12-01,"5,749.44","BBVA Bancomer, S.A.",2021-12-01 08:17:00,Roberto Carlos Gómez Ramírez,'012700028365975726,...,TRAD,Pago del día,nan,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,71681547-a00d-4b43-b366-f59ab2f7578d,Tqs,chemaesp@hotmail.com,223CB4S2K,2021-12-01,266.49,"BBVA Bancomer, S.A.",2021-12-01 08:16:28,Jose maria eparza,'012180015107748443,...,TRAD,Pago del día,nan,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [30]:
cruce_1.shape

(4838507, 24)

In [31]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
2382201,d45c5a44-707d-4697-af2c-654ab650ea02,DaniCosmetics,frezhita26@hotmail.com,9XBB5LVB2,2021-12-30,258.72,"Banco Mercantil del Norte, S.A.",2021-12-30 08:15:57,Fatima Solano,'072180011338952440,...,TRAD,Pago del día,258.72,2022-01-05 15:29:59,2022-01-05,1.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
2442243,f12bfcec-11af-4666-b01e-da6d5f14072c,Las Malditas de las 57,alvarezkristal01@gmail.com,23B3DGJ9G,2021-12-31,"2,173.64","BanCoppel, S.A.",2021-12-31 08:17:30,Rogelio Miguel Morales Rosales,'137180010029537696,...,TRAD,Pago del día,"2,173.64",2022-01-01 19:10:05,2022-01-01,1.00,ERROR,DEV.SPEICUENTA INEXISTENTE,ERROR_USUAL,NEW
2442344,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,abarrotesrr4@gmail.com,24B2DTXGK,2021-12-31,472.41,"Opciones Empresariales del Noreste, S.A. de C....",2021-12-31 21:09:58,Juana Gabriela Rodriguez Reyes,'659802020000479804,...,TRAD,Pago del día,472.41,2022-01-01 19:10:13,2022-01-01,1.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,ERROR_USUAL,NEW
2442485,e5e7483e-066d-4b56-b075-3358e7e1ddb3,Mini super zayra,rogeliogibarra@gmail.com,25QK27WLN,2021-12-31,"1,433.53","Opciones Empresariales del Noreste, S.A. de C....",2021-12-31 18:58:03,Rogelio Gaona Ibarra,'659802020000193465,...,TRAD,Pago del día,"1,433.53",2022-01-01 19:10:11,2022-01-01,1.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,ERROR_USUAL,NEW
2442691,1e16559b-a743-4443-98e5-0489cdbcdf78,ABARROTERA EL SOL,breakqz@hotmail.com,27QXVCBLD,2021-12-31,239.56,"Opciones Empresariales Del Noreste, S.A. DE C.V.",2021-12-31 21:10:00,Joao Martinez,'659802020000150738,...,TRAD,Pago del día,239.56,2022-01-01 19:10:13,2022-01-01,1.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,ERROR_USUAL,NEW


In [32]:
cruce_1_final.shape

(5813, 24)

In [33]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [34]:
duplicates_cruce_1_final.shape

(0, 24)

# CRUCE 2 MES ACTUAL

In [35]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,57622,b42fc20a-c041-4dd3-a740-3c78c85c31a5,Carniceria Local 21,RQL2J4GGS,2021-12-31,"1,348.25",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:10,1,2022,NEW,ERROR_USUAL,b42fc20a-c041-4dd3-a740-3c78c85c31a5,2022-01-01,1,Actual,TRAD,Pago del día
1,57506,12a5d534-e735-46d0-88f7-86430a745e39,Umedica,9ZSJ5KHXS,2021-12-31,143.74,ERROR,DEV.SPEICUENTA CANCELADA,2022-01-01 19:09:54,1,2022,NEW,ERROR_USUAL,12a5d534-e735-46d0-88f7-86430a745e39,2022-01-01,1,Actual,TRAD,Pago del día
2,57507,3cf78e3d-91b1-42b7-8be5-759d04f6fcba,Tienda RD,QBDJXR8NC,2021-12-31,80.90,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:14,1,2022,NEW,ERROR_USUAL,3cf78e3d-91b1-42b7-8be5-759d04f6fcba,2022-01-01,1,Actual,TRAD,Pago del día
3,57508,4dd0d5fc-7d15-4777-96e4-2dec974e7eee,Gelatto,YRSMQ5PBL,2021-12-31,"1,026.27",ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-01-01 19:09:55,1,2022,NEW,ERROR_USUAL,4dd0d5fc-7d15-4777-96e4-2dec974e7eee,2022-01-01,1,Actual,TRAD,Pago del día
4,57510,c56fedc0-285e-4cdb-83db-79aa3f683507,Frutas,XXYVQSMWX,2021-12-31,"1,290.73",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:05,1,2022,NEW,ERROR_USUAL,c56fedc0-285e-4cdb-83db-79aa3f683507,2022-01-01,1,Actual,TRAD,Pago del día


In [36]:
cruce_2.shape

(11526, 19)

In [37]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,57622,b42fc20a-c041-4dd3-a740-3c78c85c31a5,Carniceria Local 21,RQL2J4GGS,2021-12-31,"1,348.25",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:10,1,2022,NEW,ERROR_USUAL,b42fc20a-c041-4dd3-a740-3c78c85c31a5,2022-01-01,1,Actual,TRAD,Pago del día
1,57506,12a5d534-e735-46d0-88f7-86430a745e39,Umedica,9ZSJ5KHXS,2021-12-31,143.74,ERROR,DEV.SPEICUENTA CANCELADA,2022-01-01 19:09:54,1,2022,NEW,ERROR_USUAL,12a5d534-e735-46d0-88f7-86430a745e39,2022-01-01,1,Actual,TRAD,Pago del día
2,57507,3cf78e3d-91b1-42b7-8be5-759d04f6fcba,Tienda RD,QBDJXR8NC,2021-12-31,80.90,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:14,1,2022,NEW,ERROR_USUAL,3cf78e3d-91b1-42b7-8be5-759d04f6fcba,2022-01-01,1,Actual,TRAD,Pago del día
3,57508,4dd0d5fc-7d15-4777-96e4-2dec974e7eee,Gelatto,YRSMQ5PBL,2021-12-31,"1,026.27",ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-01-01 19:09:55,1,2022,NEW,ERROR_USUAL,4dd0d5fc-7d15-4777-96e4-2dec974e7eee,2022-01-01,1,Actual,TRAD,Pago del día
4,57510,c56fedc0-285e-4cdb-83db-79aa3f683507,Frutas,XXYVQSMWX,2021-12-31,"1,290.73",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:05,1,2022,NEW,ERROR_USUAL,c56fedc0-285e-4cdb-83db-79aa3f683507,2022-01-01,1,Actual,TRAD,Pago del día


In [38]:
cruce_2_final.shape

(6869, 19)

In [39]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
23,57473,22cd39fc-7966-4bad-92ca-712be05cfb1c,Telcel Voncel,X3QPK54J8,2021-12-31,80.50,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:10,1,2022,NEW,ERROR_USUAL,22cd39fc-7966-4bad-92ca-712be05cfb1c,2022-01-01,1,Actual,TRAD,Pago del día
24,57473,22cd39fc-7966-4bad-92ca-712be05cfb1c,Telcel Voncel,X3QPK54J8,2021-12-31,80.50,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:10,1,2022,NEW,ERROR_USUAL,22cd39fc-7966-4bad-92ca-712be05cfb1c,2022-01-01,1,Actual,TRAD,Pago del día
38,57494,d78de662-6c79-4c68-873e-520d67645961,Artesanas Manuales,363YZGGQD,2021-12-31,469.54,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:12,1,2022,NEW,ERROR_USUAL,d78de662-6c79-4c68-873e-520d67645961,2022-01-01,1,Actual,TRAD,Pago del día
39,57494,d78de662-6c79-4c68-873e-520d67645961,Artesanas Manuales,363YZGGQD,2021-12-31,469.54,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:12,1,2022,NEW,ERROR_USUAL,d78de662-6c79-4c68-873e-520d67645961,2022-01-01,1,Actual,TRAD,Pago del día
51,57567,af507a6f-9926-4367-bdf6-4a59ed06024f,Bambinos Estetica,XHSHFVGZZ,2021-12-31,150.92,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-01-01 19:10:11,1,2022,NEW,ERROR_USUAL,af507a6f-9926-4367-bdf6-4a59ed06024f,2022-01-01,1,Actual,TRAD,Pago del día


In [40]:
duplicates_cruce_2_final.shape

(1560, 19)

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [41]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x                 \
esquema_query                           ERROR_USUAL                  
status_cubeta                                 ERROR                  
tipo_dispersion                                  FP           TRAD   
tipo_pago                            Reprogramación Reprogramación   
0                         2022-01-01           0.00           0.00   
1                         2022-01-02           0.00           0.00   
2                         2022-01-03           0.00           0.00   
3                         2022-01-04           0.00      13,613.64   
4                         2022-01-05           0.00       3,364.30   

                                                                         \
esquema_query                                                             
status_cubeta            NEW                                              
tipo_dispersion           FP                        TRAD                  
tipo_pago       Pago del día Reprogramación Pago del día Reprogramación   
0                       0.00           0.00   352,355.92       1,216.00   
1                       0.00           0.00     8,252.42         455.16   
2                   2,168.49           0.00   130,866.37       4,594.76   
3                   3,792.71           0.00   496,113.78       5,426.50   
4                  13,917.76           0.00    41,814.36         613.27   

                            
esquema_query          All  
status_cubeta               
tipo_dispersion             
tipo_pago                   
0               353,571.92  
1                 8,707.58  
2               137,629.62  
3               518,946.63  
4                59,709.69

In [42]:
group_tipo_pago.shape

(32, 8)

In [43]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only       amount_x                              \
esquema_query                         ERROR_USUAL                               
status_cubeta                               ERROR          NEW                  
tipo_pago                          Reprogramación Pago del día Reprogramación   
0                       2022-01-01           0.00   352,355.92       1,216.00   
1                       2022-01-02           0.00     8,252.42         455.16   
2                       2022-01-03           0.00   133,034.86       4,594.76   
3                       2022-01-04      13,613.64   499,906.49       5,426.50   
4                       2022-01-05       3,364.30    55,732.12         613.27   

                          
esquema_query        All  
status_cubeta             
tipo_pago                 
0             353,571.92  
1               8,707.58  
2             137,629.62  
3             518,946.63  
4              59,709.69

In [44]:
group_esquema.shape

(32, 5)

# CRUCE 3 - MES ANTERIOR

In [45]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,748c78b8-cadc-483e-ae50-60715de44756,Wacho’s Texcoco,rafael_wachos@hotmail.com,2223TPR8N,2021-12-01,"1,097.19","BBVA Bancomer, S.A.",2021-12-01 08:15:57,Jocelyn Martinez Hernandez,'012180015634257038,...,TRAD,Pago del día,nan,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,9150491a-374e-472c-8d33-e94c9f255539,La Batucada,labatucadatoluca@gmail.com,222FZK6KC,2021-12-01,10.00,"Banco Nacional de México, S.A.",2021-12-01 08:15:57,Socorro Elizabeth Chavez Silva,'002420701716193845,...,TRAD,Pago del día,nan,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,6468a3fc-ce8a-45af-9003-bf154ae7a855,Michelle bags,ib_ago@hotmail.com,22365XV7N,2021-12-01,"2,219.28","BBVA Bancomer, S.A.",2021-12-01 08:15:57,Juan Carlos Arguelles Ochoa,'012320001871309102,...,TRAD,Pago del día,nan,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,564d2389-419a-45ec-a278-c08573ddc6de,Robe Metamorfo,robemetamorfo83@gmail.com,2238N48VN,2021-12-01,"5,749.44","BBVA Bancomer, S.A.",2021-12-01 08:17:00,Roberto Carlos Gómez Ramírez,'012700028365975726,...,TRAD,Pago del día,nan,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,71681547-a00d-4b43-b366-f59ab2f7578d,Tqs,chemaesp@hotmail.com,223CB4S2K,2021-12-01,266.49,"BBVA Bancomer, S.A.",2021-12-01 08:16:28,Jose maria eparza,'012180015107748443,...,TRAD,Pago del día,nan,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [46]:
cruce_3.shape

(4838507, 24)

In [47]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
9200,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,carmenquezada108@gmail.com,57FSM6FCT,2021-04-30,618.07,"Banco Nacional de México, S.A.",2021-12-01 08:15:57,Carmen Guadalupe Quezada Morales,'002073560111417798,...,TRAD,Reprogramación,618.07,2021-12-01 15:37:54,2021-12-01,12.00,ERROR,Cuenta cancelada,ERROR_USUAL,NEW
38350,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,carmenquezada108@gmail.com,HDBBSP5J7,2021-02-22,111.15,"Banco Nacional de México, S.A.",2021-12-01 08:15:26,Carmen Guadalupe Quezada Morales,'002073560111417798,...,TRAD,Reprogramación,111.15,2021-12-01 15:38:09,2021-12-01,12.00,ERROR,Cuenta cancelada,ERROR_USUAL,NEW
70062,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,carmenquezada108@gmail.com,VFGMG6F67,2021-02-27,354.55,"Banco Nacional de México, S.A.",2021-12-01 08:15:57,Carmen Guadalupe Quezada Morales,'002073560111417798,...,TRAD,Reprogramación,354.55,2021-12-01 15:37:54,2021-12-01,12.00,ERROR,Cuenta cancelada,ERROR_USUAL,NEW
75604,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,carmenquezada108@gmail.com,XB4QJ9WRZ,2021-02-21,178.23,"Banco Nacional de México, S.A.",2021-12-01 08:17:00,Carmen Guadalupe Quezada Morales,'002073560111417798,...,TRAD,Reprogramación,178.23,2021-12-01 15:38:03,2021-12-01,12.00,ERROR,Cuenta cancelada,ERROR_USUAL,NEW


In [48]:
cruce_3_final.shape

(4, 24)

In [49]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [50]:
duplicates_cruce_3_final.shape

(0, 24)

# CRUCE 4 - MES ANTERIOR

In [51]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,16072,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,XB4QJ9WRZ,2021-02-21,178.23,ERROR,Cuenta cancelada,2021-12-01 15:38:03,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN,TRAD,Reprogramación
1,16133,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,HDBBSP5J7,2021-02-22,111.15,ERROR,Cuenta cancelada,2021-12-01 15:38:09,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN,TRAD,Reprogramación
2,16807,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,VFGMG6F67,2021-02-27,354.55,ERROR,Cuenta cancelada,2021-12-01 15:37:54,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN,TRAD,Reprogramación
3,19196,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,57FSM6FCT,2021-04-30,618.07,ERROR,Cuenta cancelada,2021-12-01 15:37:54,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN,TRAD,Reprogramación


In [52]:
cruce_4.shape

(4, 19)

In [53]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,16072,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,XB4QJ9WRZ,2021-02-21,178.23,ERROR,Cuenta cancelada,2021-12-01 15:38:03,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN,TRAD,Reprogramación
1,16133,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,HDBBSP5J7,2021-02-22,111.15,ERROR,Cuenta cancelada,2021-12-01 15:38:09,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN,TRAD,Reprogramación
2,16807,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,VFGMG6F67,2021-02-27,354.55,ERROR,Cuenta cancelada,2021-12-01 15:37:54,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN,TRAD,Reprogramación
3,19196,ef5cb604-8970-4e94-8139-00630baea1b4,Taqueria Mision Del Real,57FSM6FCT,2021-04-30,618.07,ERROR,Cuenta cancelada,2021-12-01 15:37:54,12,2021,NEW,ERROR_USUAL,ef5cb604-8970-4e94-8139-00630baea1b4,2021-12-01,1,NaN,TRAD,Reprogramación


In [54]:
cruce_4_final.shape

(4, 19)

In [55]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago


In [56]:
duplicates_cruce_4_final.shape

(0, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [57]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ultimo_mov_date_only       amount_x         
esquema_query                           ERROR_USUAL      All
status_cubeta                                   NEW         
tipo_dispersion                                TRAD         
tipo_pago                            Reprogramación         
0                         2021-12-01       1,262.00 1,262.00
1                                All       1,262.00 1,262.00

In [58]:
group_tipo_pago_last_month.shape

(2, 3)

In [59]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ultimo_mov_date_only         amount         
esquema_query                         ERROR_USUAL      All
status_cubeta                                 NEW         
tipo_pago                          Reprogramación         
0                       2021-12-01       1,262.00 1,262.00
1                              All       1,262.00 1,262.00

In [60]:
group_esquema_last_month.shape

(2, 3)

In [62]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL ENERO 2022.xlsx', engine='xlsxwriter', options={'encoding':'utf-8'})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.save()
writer.close()